# ODBC. Connection to databases 

## Connection to PostgreSQL server with Psycopg2
Installation Psycopg2: pip install psycopg2 (psycopg2==2.7.7)

## Creating the connection to PostgreSQL database

In [5]:
import psycopg2
from psycopg2 import Error
try:
    
    #usr,password,host_ = 'postgres','root','localhost'
    usr,password,host_ = 'asgeusr@mypostgrestest','jdjfR3eM','mypostgrestest.postgres.database.azure.com'
    connection = psycopg2.connect(user = usr,
                                  password = password,
                                  host = host_,
                                  port = "5432",
                                  database = "Interns")
    
    cursor = connection.cursor() #Handles the connection

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while opening", error)

Error while opening could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "mypostgrestest.postgres.database.azure.com" (23.99.34.75) and accepting
	TCP/IP connections on port 5432?



## Close the connection

In [2]:
#closing database connection.
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

PostgreSQL connection is closed


## Query data

#### Creating a new table

In [14]:
createTable = '''create table InternsInformation(
                    id varchar(15),
                    username varchar(4),
                    name varchar(30),
                    lastname varchar(40),
                    age integer
                );'''

cursor.execute(createTable)
connection.commit()
print("Table created successfully in PostgreSQL ")

Table created successfully in PostgreSQL 


#### Inserting data into a table

In [15]:
# the data set
interns = [
    ('402310453','asge','Andres','Gutierrez',23),
    ('123456790','cope','Carlos','Portuguez',22),
    ('546413431','dima','David','Morales',22),
    ('189146131','waee','William','Espinoza',22),
]
for intern in interns:
    insert = (f"insert into InternsInformation (id,username,name,lastname,age)" 
              f"values (\'{intern[0]}\',\'{intern[1]}\',\'{intern[2]}\',\'{intern[3]}\',{intern[4]});")
    cursor.execute(insert)
    connection.commit()
    count = cursor.rowcount
    print (count, "Record inserted successfully into InternsInformation table")

1 Record inserted successfully into InternsInformation table
1 Record inserted successfully into InternsInformation table
1 Record inserted successfully into InternsInformation table
1 Record inserted successfully into InternsInformation table


#### Select data from a table

In [4]:
# the SQL QUERY
postgreSQL_select_Query = "select * from InternsInformation"

cursor.execute(postgreSQL_select_Query)
print("Selecting rows from InternsInformation table using cursor.fetchall")
records = cursor.fetchall() 

print("Print each row and it's columns values")
for row in records:
   print(row)

# As you can see, it returns a tuple, so we can use all the built-in methods for tuples (e.g. [])

InterfaceError: cursor already closed

## Lets create a dataframe with Pandas

In [75]:
import pandas as pd

## Executing a stored procedured

In [6]:
# insert_intern procedure perform an insert of a intern into the InternsInformation table
# and prints the inserted value

#Buscar como se crea un SP
sp = (f" CREATE OR REPLACE FUNCTION insert_intern (  idp varchar(15),"
        f" usernamep varchar(4),"
        f" namep varchar(30),"
        f" lastnamep varchar(40),"
        f" agep integer) "
        f" RETURNS TABLE (idr varchar(15),usernamer varchar(4),namer varchar(30),lastnamer varchar(40),ager integer) "
        f" AS $$"
        f" BEGIN"
        f" insert into InternsInformation values (idp,usernamep,namep,lastnamep,agep);"
        f" RETURN QUERY "
        f" SELECT id,username,name,lastname,age FROM InternsInformation WHERE InternsInformation.id = idp ;"
        f" END; $$ "
        f" LANGUAGE \'plpgsql\';")

cursor.execute(sp)
connection.commit()

#### Calling the procedure

In [7]:
cursor.callproc('insert_intern',('110055252', 'mlsa', 'Melissa', 'Cardenas', 21,))       
row = cursor.fetchone()
while row is not None:
    print(row)
    row = cursor.fetchone()

('110055252', 'mlsa', 'Melissa', 'Cardenas', 21)
